# 분석환경 준비

In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np
import random
import tensorflow as tf

# 랜덤 시드 고정
SEED=12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)  
print("시드 고정: ", SEED)

시드 고정:  12


In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 데이터 전처리

In [ ]:
# 데이콘 사이트에서 다운로드한 CSV파일을 읽어오기
drive_path = "/gdrive/My Drive/"

train = pd.read_csv(drive_path + "wine/train.csv")
test = pd.read_csv(drive_path + "wine/test.csv")
submission = pd.read_csv(drive_path + "wine/sample_submission.csv")

print(train.shape, test.shape, submission.shape)

(5497, 14) (1000, 13) (1000, 2)


In [ ]:
train.head(2)

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red


In [ ]:
submission.head()

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
train['type'].value_counts()

white    4159
red      1338
Name: type, dtype: int64

In [ ]:
train['type'] = np.where(train['type']=='white', 1, 0).astype(int)
test['type'] = np.where(test['type']=='white', 1, 0).astype(int)
train['type'].value_counts()

1    4159
0    1338
Name: type, dtype: int64

In [ ]:
train['quality'].value_counts()

6    2416
5    1788
7     924
4     186
8     152
3      26
9       5
Name: quality, dtype: int64

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train.loc[:, 'quality'] - 3)
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# 피처 선택
X_train = train.loc[:, 'fixed acidity':]
X_test = test.loc[:, 'fixed acidity':]

# 피처 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

print(X_train_scaled.shape, y_train.shape)
print(X_test_scaled.shape)

(5497, 12) (5497, 7)
(1000, 12)


# 신경망 학습

In [ ]:
# 심층 신경망 모델
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(train_data, train_target):
    model = Sequential()
    model.add(Dense(128, activation='tanh', input_dim=train_data.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(train_target.shape[1], activation='softmax'))

    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', 
                metrics=['acc', 'mae'])

    return model

model = build_model(X_train_scaled, y_train)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1664      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 231       
Total params: 12,231
Trainable params: 12,231
Non-trainable params: 0
____________________________________________________

# 콜백 함수

In [ ]:
# Early Stopping 기법
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.15, 
                                            shuffle=True, random_state=SEED)

early_stopping = EarlyStopping(monitor='val_loss',  patience=10)
history = model.fit(X_tr, y_tr, batch_size=64, epochs=200,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],                    
                    verbose=2)

Epoch 1/200
73/73 - 0s - loss: 1.3005 - acc: 0.4574 - mae: 0.1930 - val_loss: 1.1688 - val_acc: 0.5055 - val_mae: 0.1808
Epoch 2/200
73/73 - 0s - loss: 1.1752 - acc: 0.5015 - mae: 0.1779 - val_loss: 1.1034 - val_acc: 0.5430 - val_mae: 0.1732
Epoch 3/200
73/73 - 0s - loss: 1.1449 - acc: 0.5169 - mae: 0.1742 - val_loss: 1.0814 - val_acc: 0.5442 - val_mae: 0.1681
Epoch 4/200
73/73 - 0s - loss: 1.1285 - acc: 0.5210 - mae: 0.1719 - val_loss: 1.1267 - val_acc: 0.5042 - val_mae: 0.1698
Epoch 5/200
73/73 - 0s - loss: 1.1177 - acc: 0.5330 - mae: 0.1709 - val_loss: 1.0708 - val_acc: 0.5564 - val_mae: 0.1680
Epoch 6/200
73/73 - 0s - loss: 1.1098 - acc: 0.5240 - mae: 0.1712 - val_loss: 1.0613 - val_acc: 0.5539 - val_mae: 0.1667
Epoch 7/200
73/73 - 0s - loss: 1.1028 - acc: 0.5291 - mae: 0.1704 - val_loss: 1.0571 - val_acc: 0.5503 - val_mae: 0.1658
Epoch 8/200
73/73 - 0s - loss: 1.0996 - acc: 0.5216 - mae: 0.1700 - val_loss: 1.0612 - val_acc: 0.5576 - val_mae: 0.1655
Epoch 9/200
73/73 - 0s - loss: 1

In [ ]:
model.evaluate(X_val, y_val)

26/26 [==============================] - 0s 1ms/step - loss: 1.0265 - acc: 0.5576 - mae: 0.1614


[1.0264517068862915, 0.5575757622718811, 0.16143734753131866]

In [ ]:
# test 데이터에 대한 예측값 정리
y_pred_proba = model.predict(X_test_scaled)
y_pred_proba[:5]

array([[9.47426248e-04, 6.51338184e-03, 1.07454561e-01, 6.22603416e-01,
        2.17842266e-01, 4.44285981e-02, 2.10311497e-04],
       [3.37807485e-03, 2.81181000e-02, 8.08466911e-01, 1.52429342e-01,
        7.13104848e-03, 4.59584000e-04, 1.69139166e-05],
       [5.04492735e-03, 1.34563502e-02, 4.53614473e-01, 4.81525809e-01,
        3.60224396e-02, 1.01609509e-02, 1.75175010e-04],
       [3.16306017e-03, 2.47096531e-02, 4.85043049e-01, 4.44753528e-01,
        3.69130187e-02, 5.17660752e-03, 2.41071990e-04],
       [1.51876360e-03, 2.17930879e-03, 1.87265258e-02, 2.98122764e-01,
        5.55454612e-01, 1.21920586e-01, 2.07743281e-03]], dtype=float32)

In [ ]:
y_pred_label = np.argmax(y_pred_proba, axis=-1) + 3
y_pred_label[:5]

array([6, 5, 6, 5, 7])

In [ ]:
# 제출양식에 맞게 정리
submission['quality'] = y_pred_label.astype(int)
submission.head()

,index,quality
0,0,6
1,1,5
2,2,6
3,3,5
4,4,7


In [ ]:
# 제출파일 저장    
submission.to_csv(drive_path + "wine/wine_dnn_001.csv", index=False)   